# Lab 3: Singing a song
## Introduction
The song “12 Days of Christmas”, written around 1780, tells the tale of many gifts a person receives in the days leading up to Christmas (link to lyrics.

Note
You can watch a video of the 12 Days of Christmas at the Cambria Christmas Market.

These gifts repeat and compound; on the first day, the narrator receives

A partridge in a pear tree.
On the twelfth day, they receive

Twelve Drummers Drumming
Eleven Pipers Piping
Ten Lords a Leaping
Nine Ladies Waiting
Eight Maids a Milking
Seven Swans a Swimming
Six Geese a Laying
Five Golden Rings
Four Calling Birds
Three French Hens
Two Turtle Doves
And a Partridge in a Pear Tree
This week, your task will be to write functions that automatically sing this very repetitive song.

### Data set

Run the code provided to load in a data set called xmas that contains the crucial information about the gifts in the song. We will use this data set to test out our functions as we work on them.

In [207]:
import pandas as pd
import numpy as np
from num2words import num2words
xmas = pd.read_csv("https://www.dropbox.com/scl/fi/qxaslqqp5p08i1650rpc4/xmas.csv?rlkey=erdxi7jbh7pqf9fh4lv4cayp5&dl=1")

# Function 1: pluralize_gift()

The gifts are listed in singular: for example, on day five the narrator receives “five golden rings”, but the entry in the data set for the gift on day five simply says “ring”.

Hint 1: The gifts on days six and nine have unusual pluralization. You may assume that in other data sets, there will be no additional special cases besides these types.

Hint 2: The following small code snippets may be useful to you

In [208]:
def pluralize_gift(gift):
  """
  Returns plural of a noun
  
  Parameters
  ----------
  gift: str
    A noun
    
  Return
  ------
  str
    Plural version
  """


  if gift.find("oo") != -1:
    gift = gift.replace("oo", "ee")
  elif gift[-1] == "y":
    gift = gift.replace("y", "ies")
  else:
    gift = gift + "s"

  return gift




# Test your Function

In [209]:
# Should work
pluralize_gift("goose")

'geese'

In [197]:
# Will work if your function is vectorized! 
#pluralize_gift = np.vectorize(pluralize_gift)
pluralize_gift(xmas['Gift.Item'])

AttributeError: 'Series' object has no attribute 'find'

# Function 2: make_phrase()
Write a function called make_phrase() that takes as input the necessary information, and returns a phrase. For example,

make_phrase(num_word = "ten", 
            item = "lords", 
            verb = "a-leaping", 
            adjective = "", 
            location = "")
should return

"ten lords a-leaping"

In [248]:
def make_phrase(num, num_word, item, verb, adjective, location):
  """
  Returns full phrase
  
  Parameters
  ----------
  num: int
    A number
  num_word: str
    a number in words
  item: str
    a noun
  verb: str
    a verb
  adjective: str
    an adjective
  location: str
    a location
    
  Return
  ------
  str
    Combined phrase
  """
  num_dict = {
        1: "One",
        2: "Two",
        3: "Three",
        4: "Four",
        5: "Five",
        6: "Six",
        7: "Seven",
        8: "Eight",
        9: "Nine",
        10: "Ten",
        11: "Eleven",
        12: "Twelve"
    }

  ## Step 1: Replace NAs with blank strings
  verb = pd.Series([verb]).fillna("").iloc[0]
  adjective = pd.Series([adjective]).fillna("").iloc[0]
  location = pd.Series([location]).fillna("").iloc[0]
  
  ## Step 2: If the day number is larger than 1, the gift items need pluralized!
  if num > 1:
    item = pluralize_gift(item)
    #num = num2words(num)
  else:
    item
  
  ## Step 3: Figure out if a gift item starts with a vowel
  ## Step 4: For the first day, if the gift item starts with a vowel, 
  # replace the day with "an" and if the gift item does not start with a vowel, 
  # replace the day with "a" (e.g. a partridge in a pear tree). 
  # If it is not the first day, use just the number word (e.g. ten lords a leap)
  
  vowel = ["a", "e", "i", "o", "u"]
  if (num == 1) & (item[0].lower() in vowel):
    num = "an"
  elif (num == 1):
    num = "a"
  else:
    num = num2words(num)


  ## Step 5: Put all of the pieces together into one string and return!

  #words = [article, str(num), adjective, item, verb, location]
  #phrase = " ".join(filter(None, words)).strip()

  phrase = f"{str(num)} {adjective} {item} {verb} {location}".strip()
  phrase = " ".join(phrase.split())

  return phrase




# Test Your Function
Make sure to try your function out on small examples and on the xmas data.

Then, use the function to make a new column of the xmas column called Full.Phrase containing the sentences for the new gift on that day.

In [251]:
# test function

make_phrase(num = 2, num_word = "second", item = "dove", verb = "", adjective = "turtle", location = "")

'two turtle doves'

In [249]:
xmas["Full.Phrase"] = xmas.apply(lambda row: make_phrase(num = row["Day"], 
                                                         num_word = row["Day.in.Words"],
                                                         item = row["Gift.Item"],
                                                         verb = row["Verb"],
                                                         adjective = row["Adjective"],
                                                         location = row["Location"]), axis = 1)



# Function 3: sing_day()
Write a function called sing_day() that takes as input:

A dataset (input as a dataframe)

A number indicating which day to sing about (input as an integer)

The name of a column in the dataset that contains the phrases for each day (input as an tidy name)

For example,

sing_day(xmas, 2, Full.Phrase)
should return

On the second day of Christmas, my true love sent to me:
two turtle doves and
a partridge in a pear tree.


In [336]:
def sing_day(dataset, num, phrase_col):
  """
  Returns song until specified day
  
  Parameters
  ----------
  dataset: 
    
  num: int
    
  phrase_col: 

  Return
  ------
  str
    Combined phrase
  """
  
  # Step 1: Setup the intro line
  num_word = num2words(num, to = "ordinal")  # convert "1" to "first" etc.
  intro = "On the " + num_word + " day of Christmas, my true love sent to me:\n"
  
  # Step 2: Sing the gift phrases
  # Hint: What order are they gifts sung in each day?
  gifts = ""
  for i in range(num - 1, -1, -1):
    gifts += dataset.iloc[i][phrase_col] + "\n"
    if i == 1:
      gifts += "and "
  
  # Step 3: Put it all together and return
  return print(intro + gifts)

In [337]:
sing_day(xmas, 6, "Full.Phrase")

On the sixth day of Christmas, my true love sent to me:
six geese a-laying
five golden rings
four calling birds
three french hens
two turtle doves
and a partridge in a pear tree

On the sixth day of Christmas, my true love sent to me:
six geese a-laying
five golden rings
four calling birds
three french hens
two turtle doves
and a partridge in a pear tree



# Appendix

num2words function: https://pypi.org/project/num2words/ 
The num2words function was used to convert the Day integers into written words

for a decreasing for loop:
https://stackoverflow.com/questions/4294082/decreasing-for-loops-in-python-impossible 

In [293]:
n = 5
for i in range(n - 1, -1, -1):
    print(i)


4
3
2
1
0
4
3
2
1
0
